In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simple_moving_average(df, column, window):
    df[f'{column}_SMA_{window}'] = df[column].rolling(window=window).mean()
    return df

# Пример использования:
# df = simple_moving_average(df, 'sales', window=3)


def weighted_moving_average(df, column, window):
    weights = np.arange(1, window + 1)  # Веса от 1 до размера окна
    wma = df[column].rolling(window).apply(lambda x: np.dot(x, weights)/weights.sum(), raw=True)
    df[f'{column}_WMA_{window}'] = wma
    return df

# Пример использования:
# df = weighted_moving_average(df, 'sales', window=3)

def exponential_moving_average(df, column, span):
    df[f'{column}_EMA_{span}'] = df[column].ewm(span=span, adjust=False).mean()
    return df

# Пример использования:
# df = exponential_moving_average(df, 'sales', span=3)

def line_plot_with_legend(df, variables):
    # Построение линий для каждой переменной
    plt.figure(figsize=(10, 6))
    
    for var in variables:
        sns.lineplot(data=df, x=df.index, y=var, label=var)
    
    # Подписи осей
    plt.xlabel('Index')
    plt.ylabel('Values')
    
    # Добавляем легенду
    plt.legend(title="Variables")
    
    # Отображаем график
    plt.tight_layout()
    plt.show()

import pandas as pd

def calculate_mape(df, target_col, predictions_list, add=1e-10):
    """
    Рассчитывает MAPE для нескольких прогнозов по отношению к таргету.
    
    Параметры:
    - df: DataFrame с данными.
    - target_col: колонка с истинными значениями (таргет).
    - predictions_list: список названий столбцов с прогнозами.
    - add: значение, добавляемое к таргету для избежания деления на ноль.
    
    Возвращает:
    - DataFrame с названиями предсказаний и их значениями MAPE.
    """
    
    # Проверка, что таргет и прогнозы есть в DataFrame
    if target_col not in df.columns:
        raise ValueError(f"Колонка с таргетом '{target_col}' не найдена в DataFrame.")
    
    missing_predictions = [pred for pred in predictions_list if pred not in df.columns]
    if missing_predictions:
        raise ValueError(f"Прогнозы {missing_predictions} не найдены в DataFrame.")
    
    # Список для хранения MAPE для каждого прогноза
    mape_results = []
    
    # Рассчет MAPE для каждого прогноза
    for pred_col in predictions_list:
        # MAPE = (1/n) * sum(|(y_true - y_pred)| / (y_true + add)) * 100
        mape = (abs(df[target_col] - df[pred_col]) / (df[target_col] + add)).mean() * 100
        mape_results.append({'Prediction': pred_col, 'MAPE': mape})
    
    # Возвращаем результаты в виде DataFrame
    return pd.DataFrame(mape_results)

def create_lag_features_with_prediction(df, feature_list, min_lag, max_lag):
    """
    Создает лаговые признаки для обучения и для предсказания на следующий день.

    Параметры:
    - df: DataFrame с исходными данными.
    - feature_list: список колонок, для которых создаются лаги.
    - min_lag: минимальный лаг.
    - max_lag: максимальный лаг.

    Возвращает:
    - lagged_df: DataFrame для обучения, содержащий лаговые признаки и целевую переменную.
    - prediction_df: DataFrame с одной строкой, содержащей лаговые признаки для предсказания на следующий день.
    """
    # Создаем лаговые признаки для обучения
    lagged_features = []
    for lag in range(min_lag, max_lag + 1):
        lagged = df[feature_list].shift(lag).add_suffix(f'_lag_{lag}')
        lagged_features.append(lagged)

    # Объединяем исходные данные с лаговыми признаками
    lagged_df = pd.concat([df] + lagged_features, axis=1)
    lagged_df.dropna(inplace=True)

    # Создаем лаговые признаки для предсказания на следующий день
    prediction_dict = {}
    for feature in feature_list:
        for lag in range(min_lag, max_lag + 1):
            lag_feature_name = f'{feature}_lag_{lag}'
            # Получаем значение для соответствующего лага
            prediction_dict[lag_feature_name] = df[feature].iloc[-lag]

    prediction_df = pd.DataFrame([prediction_dict])

    return lagged_df, prediction_df


In [3]:
shop_sales = pd.read_csv('shop_sales.csv')
shop_sales_dates = pd.read_csv('shop_sales_dates.csv')
shop_sales_prices = pd.read_csv('shop_sales_prices.csv')

In [4]:
# all in one
def merge_files_to_dataset(
        shop_sales: pd.DataFrame,
        shop_sales_prices: pd.DataFrame,
        shop_sales_dates: pd.DataFrame
) -> pd.DataFrame:
    shop_sales_dates['date_id'] = shop_sales_dates.index + 1
    merged = shop_sales.merge(shop_sales_dates, on='date_id', how='left')
    #merged.fillna('Nothing', inplace=True)
    main_df = merged.merge(shop_sales_prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
    main_df['sell_price'] = main_df['sell_price'].ffill().bfill()
    return main_df

In [5]:
df = merge_files_to_dataset(shop_sales, shop_sales_prices, shop_sales_dates)

In [6]:
df.columns

Index(['item_id', 'store_id', 'date_id', 'cnt', 'date', 'wm_yr_wk', 'weekday',
       'wday', 'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2', 'CASHBACK_STORE_1', 'CASHBACK_STORE_2',
       'CASHBACK_STORE_3', 'sell_price'],
      dtype='object')

In [7]:
def summ_sales_data(df, date_column, granularity):

    df[date_column] = pd.to_datetime(df[date_column])
    if granularity == 'Day':
        return df
    elif granularity == 'Week':
        resample_rule = 'W' 
    elif granularity == 'Month':
        resample_rule = 'M'
    
    # Устанавливаем индекс на 'date_column' и 'item_id', чтобы выполнить группировку по обоим уровням
    df.set_index([date_column, 'item_id'], inplace=True)
    
    # Группируем по дате с учетом resample_rule и сохраняем группировку по 'item_id'
    grouped = df.groupby(['item_id']).resample(resample_rule, level=0).agg({
        'cnt': 'sum',
        'sell_price': 'mean',
        'CASHBACK_STORE_1': lambda x: x.sum() / len(x),
        'CASHBACK_STORE_2': lambda x: x.sum() / len(x),
        'CASHBACK_STORE_3': lambda x: x.sum() / len(x),
        'store_id': 'first',
        'date_id': 'first',  
        'wm_yr_wk': 'first',  
        'weekday': 'first',  
        'wday': 'first',  
        'month': 'first',  
        'year': 'first',  
        'event_name_1': lambda x: x.dropna().iloc[0] if not x.dropna().empty else None,
        'event_type_1': lambda x: x.dropna().iloc[0] if not x.dropna().empty else None,  
        'event_name_2': lambda x: x.dropna().iloc[0] if not x.dropna().empty else None,
        'event_type_2': lambda x: x.dropna().iloc[0] if not x.dropna().empty else None,
    })
    
    grouped_df = grouped.reset_index()

    return grouped_df

In [8]:
df = summ_sales_data(df, 'date', granularity = 'Day')

In [9]:
df[df['item_id'] == 'STORE_1_064']

,item_id,store_id,date_id,cnt,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,CASHBACK_STORE_1,CASHBACK_STORE_2,CASHBACK_STORE_3,sell_price
54570,STORE_1_064,STORE_1,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,2.54
54571,STORE_1_064,STORE_1,2,1,2011-01-30,11101,Sunday,2,1,2011,NaN,NaN,NaN,NaN,0,0,0,2.54
54572,STORE_1_064,STORE_1,3,0,2011-01-31,11101,Monday,3,1,2011,NaN,NaN,NaN,NaN,0,0,0,2.54
54573,STORE_1_064,STORE_1,4,0,2011-02-01,11101,Tuesday,4,2,2011,NaN,NaN,NaN,NaN,0,1,1,2.54
54574,STORE_1_064,STORE_1,5,0,2011-02-02,11101,Wednesday,5,2,2011,NaN,NaN,NaN,NaN,1,1,0,2.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56384,STORE_1_064,STORE_1,1815,0,2016-01-17,11551,Sunday,2,1,2016,NaN,NaN,NaN,NaN,0,0,0,2.68
56385,STORE_1_064,STORE_1,1816,0,2016-01-18,11551,Monday,3,1,2016,MartinLutherKingDay,National,NaN,NaN,0,0,0,2.68
56386,STORE_1_064,STORE_1,1817,0,2016-01-19,11551,Tuesday,4,1,2016,NaN,NaN,NaN,NaN,0,0,0,2.68
56387,STORE_1_064,STORE_1,1818,1,2016-01-20,11551,Wednesday,5,1,2016,NaN,NaN,NaN,NaN,0,0,0,2.68


In [10]:
df.sort_values('date')

,item_id,store_id,date_id,cnt,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,CASHBACK_STORE_1,CASHBACK_STORE_2,CASHBACK_STORE_3,sell_price
0,STORE_2_085,STORE_2,1,3,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,1.00
78217,STORE_1_714,STORE_1,1,47,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,1.48
3638,STORE_2_054,STORE_2,1,6,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,2.28
56389,STORE_1_065,STORE_1,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,2.94
25466,STORE_2_716,STORE_2,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,1.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27284,STORE_2_716,STORE_2,1819,0,2016-01-21,11551,Thursday,6,1,2016,NaN,NaN,NaN,NaN,0,0,0,1.28
30922,STORE_3_114,STORE_3,1819,4,2016-01-21,11551,Thursday,6,1,2016,NaN,NaN,NaN,NaN,0,0,0,2.73
21827,STORE_2_587,STORE_2,1819,17,2016-01-21,11551,Thursday,6,1,2016,NaN,NaN,NaN,NaN,0,0,0,2.48
7275,STORE_2_325,STORE_2,1819,3,2016-01-21,11551,Thursday,6,1,2016,NaN,NaN,NaN,NaN,0,0,0,1.25


In [11]:
def df_encoding(sku: pd.DataFrame) -> pd.DataFrame:
    df_one_hot = sku.copy()
    columns = ['event_name_1', 'event_type_1',	'event_name_2',	'event_type_2', 'weekday']
    # Применить one-hot encoding для указанных столбцов
    df_one_hot = pd.get_dummies(df_one_hot, columns=columns, dtype=int)
    return df_one_hot

In [12]:
df = df_encoding(df)

In [13]:
df.columns

Index(['item_id', 'store_id', 'date_id', 'cnt', 'date', 'wm_yr_wk', 'wday',
       'month', 'year', 'CASHBACK_STORE_1', 'CASHBACK_STORE_2',
       'CASHBACK_STORE_3', 'sell_price', 'event_name_1_Chanukah End',
       'event_name_1_Christmas', 'event_name_1_Cinco De Mayo',
       'event_name_1_ColumbusDay', 'event_name_1_Easter',
       'event_name_1_Eid al-Fitr', 'event_name_1_EidAlAdha',
       'event_name_1_Father's day', 'event_name_1_Halloween',
       'event_name_1_IndependenceDay', 'event_name_1_LaborDay',
       'event_name_1_LentStart', 'event_name_1_LentWeek2',
       'event_name_1_MartinLutherKingDay', 'event_name_1_MemorialDay',
       'event_name_1_Mother's day', 'event_name_1_NBAFinalsEnd',
       'event_name_1_NBAFinalsStart', 'event_name_1_NewYear',
       'event_name_1_OrthodoxChristmas', 'event_name_1_OrthodoxEaster',
       'event_name_1_Pesach End', 'event_name_1_PresidentsDay',
       'event_name_1_Purim End', 'event_name_1_Ramadan starts',
       'event_name_1_StPa

In [14]:
df = simple_moving_average(df, 'cnt', 3)
df = weighted_moving_average(df, 'cnt', 3)
df = exponential_moving_average(df, 'cnt', 3)

In [15]:
df.columns

Index(['item_id', 'store_id', 'date_id', 'cnt', 'date', 'wm_yr_wk', 'wday',
       'month', 'year', 'CASHBACK_STORE_1', 'CASHBACK_STORE_2',
       'CASHBACK_STORE_3', 'sell_price', 'event_name_1_Chanukah End',
       'event_name_1_Christmas', 'event_name_1_Cinco De Mayo',
       'event_name_1_ColumbusDay', 'event_name_1_Easter',
       'event_name_1_Eid al-Fitr', 'event_name_1_EidAlAdha',
       'event_name_1_Father's day', 'event_name_1_Halloween',
       'event_name_1_IndependenceDay', 'event_name_1_LaborDay',
       'event_name_1_LentStart', 'event_name_1_LentWeek2',
       'event_name_1_MartinLutherKingDay', 'event_name_1_MemorialDay',
       'event_name_1_Mother's day', 'event_name_1_NBAFinalsEnd',
       'event_name_1_NBAFinalsStart', 'event_name_1_NewYear',
       'event_name_1_OrthodoxChristmas', 'event_name_1_OrthodoxEaster',
       'event_name_1_Pesach End', 'event_name_1_PresidentsDay',
       'event_name_1_Purim End', 'event_name_1_Ramadan starts',
       'event_name_1_StPa

In [16]:
id = ['STORE_1_714']

In [17]:
df_m = df[df['item_id'].isin(id)]

In [18]:
df_m

,item_id,store_id,date_id,cnt,date,wm_yr_wk,wday,month,year,CASHBACK_STORE_1,...,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,cnt_SMA_3,cnt_WMA_3,cnt_EMA_3
78217,STORE_1_714,STORE_1,1,47,2011-01-29,11101,1,1,2011,0,...,0,0,1,0,0,0,0,38.000000,41.166667,40.662406
78218,STORE_1_714,STORE_1,2,38,2011-01-30,11101,2,1,2011,0,...,0,0,0,1,0,0,0,41.333333,41.166667,39.331203
78219,STORE_1_714,STORE_1,3,28,2011-01-31,11101,3,1,2011,0,...,0,1,0,0,0,0,0,37.666667,34.500000,33.665602
78220,STORE_1_714,STORE_1,4,27,2011-02-01,11101,4,2,2011,0,...,0,0,0,0,0,1,0,31.000000,29.166667,30.332801
78221,STORE_1_714,STORE_1,5,23,2011-02-02,11101,5,2,2011,1,...,0,0,0,0,0,0,1,26.000000,25.166667,26.666400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80031,STORE_1_714,STORE_1,1815,18,2016-01-17,11551,2,1,2016,0,...,0,0,0,1,0,0,0,17.666667,18.000000,17.619902
80032,STORE_1_714,STORE_1,1816,10,2016-01-18,11551,3,1,2016,0,...,0,1,0,0,0,0,0,15.666667,14.166667,13.809951
80033,STORE_1_714,STORE_1,1817,12,2016-01-19,11551,4,1,2016,0,...,0,0,0,0,0,1,0,13.333333,12.333333,12.904975
80034,STORE_1_714,STORE_1,1818,11,2016-01-20,11551,5,1,2016,0,...,0,0,0,0,0,0,1,11.000000,11.166667,11.952488


In [19]:
df_m.columns

Index(['item_id', 'store_id', 'date_id', 'cnt', 'date', 'wm_yr_wk', 'wday',
       'month', 'year', 'CASHBACK_STORE_1', 'CASHBACK_STORE_2',
       'CASHBACK_STORE_3', 'sell_price', 'event_name_1_Chanukah End',
       'event_name_1_Christmas', 'event_name_1_Cinco De Mayo',
       'event_name_1_ColumbusDay', 'event_name_1_Easter',
       'event_name_1_Eid al-Fitr', 'event_name_1_EidAlAdha',
       'event_name_1_Father's day', 'event_name_1_Halloween',
       'event_name_1_IndependenceDay', 'event_name_1_LaborDay',
       'event_name_1_LentStart', 'event_name_1_LentWeek2',
       'event_name_1_MartinLutherKingDay', 'event_name_1_MemorialDay',
       'event_name_1_Mother's day', 'event_name_1_NBAFinalsEnd',
       'event_name_1_NBAFinalsStart', 'event_name_1_NewYear',
       'event_name_1_OrthodoxChristmas', 'event_name_1_OrthodoxEaster',
       'event_name_1_Pesach End', 'event_name_1_PresidentsDay',
       'event_name_1_Purim End', 'event_name_1_Ramadan starts',
       'event_name_1_StPa

In [28]:
from Tools import *
# import metrics for var selection
from sklearn.metrics import mean_absolute_percentage_error as MAPE

In [ ]:
df_m  = df[df['item_id'].isin([list_sku])].copy()
df_m = simple_moving_average(df_m, 'cnt', 3)
df_m = weighted_moving_average(df_m, 'cnt', 3)
df_m = exponential_moving_average(df_m, 'cnt', 3)

technical_list = ['index_time', 'item_id', 'store_id', 'date_id','cnt', 'date', 'wm_yr_wk', 'wday']
list_for_lags  = [i for i in df_m.columns if i not in technical_list]
lags, pred_df = create_lag_features_with_prediction(df_m, list_for_lags, horizont, horizont)

In [29]:
def get_preds(df, list_sku, horizont):
    
    df_m  = df[df['item_id'].isin(list_sku)].copy()
    df_m = simple_moving_average(df_m, 'cnt', 3)
    df_m = weighted_moving_average(df_m, 'cnt', 3)
    df_m = exponential_moving_average(df_m, 'cnt', 3)
    
    technical_list = ['index_time', 'item_id', 'store_id', 'date_id','cnt', 'date', 'wm_yr_wk', 'wday']
    list_for_lags  = [i for i in df_m.columns if i not in technical_list]
    lags, pred_df = create_lag_features_with_prediction(df_m, list_for_lags, horizont, horizont)
    lags_cols = [i for i in lags.columns if i not in df_m.columns]
    df_m = pd.concat([df_m, lags[lags_cols]], axis = 1)
    # Задаем значение для генератора случайных чисел
    seed_value = 23
    np.random.seed(seed_value)
    
    # creating cross validator
    window_m = len(df_m.date.unique())//5
    test_m = window_m//5
    cv_datetime = DateTimeSeriesSplit(window = window_m, n_splits= 4, test_size = test_m, margin=0)
    group_dt = df_m['date']
    
    # create model for selector
    from lightgbm import LGBMRegressor
    model = LGBMRegressor(max_depth=3, verbosity = -1)
    # create selector
    selector1 = Kraken(model, cv_datetime, MAPE, 'exp1')

    # get rank dict from vars
    selector1.get_rank_dict(df_m, df_m['cnt'], lags_cols, group_dt)

    ## get vars
    vars_final = selector1.get_vars(df_m, df_m['cnt'], early_stopping_rounds = 100, group_dt = group_dt)
    
    if len(vars_final) == 0:
        vars_final = [i for i in lags_cols if i.startswith('cnt')]
    
    test_dates = pd.Series(df_m['date'].unique()).sort_values().tail(max(3, test_m//2)).values
    
    X_train = df_m[~df_m['date'].isin(test_dates)]
    y_train = df_m[~df_m['date'].isin(test_dates)]['cnt']

    # oot - out of time
    X_oot = df_m[df_m['date'].isin(test_dates)]
    #y_oot = df_m[df_m['date'].isin(test_dates)]['cnt']
    
    model.fit(X_train[vars_final], y_train)
    pred_df['date'] = X_oot.date.max() + pd.Timedelta(days=horizont)
    pred_df['cnt'] = np.nan
    pred_df['item_id'] = X_oot.item_id.max()
    pred_df['mean'] = X_oot.cnt.mean()
    X_oot['mean'] = X_oot.cnt.mean()
    list_cont = vars_final + ['mean','cnt_SMA_3_lag_1','item_id', 'date','cnt']
    data_prediction = pd.concat([X_oot[list_cont], pred_df[list_cont]], axis = 0)
    data_prediction['model_prediction'] = model.predict(data_prediction[vars_final])
    return data_prediction


In [30]:
data_prediction = get_preds(df, ['STORE_1_714'], 1)

запуск первого шага
new var_for_add ! month_lag_1
едем дальше
в итоге получили список ['month_lag_1']
запуск первого шага
new var_for_add ! cnt_EMA_3_lag_1
new var_for_add ! cnt_WMA_3_lag_1
едем дальше
в итоге получили список ['month_lag_1', 'cnt_WMA_3_lag_1']
запуск первого шага
new var_for_add ! weekday_Friday_lag_1
едем дальше
в итоге получили список ['month_lag_1', 'cnt_WMA_3_lag_1', 'weekday_Friday_lag_1']
запуск первого шага
new var_for_add ! year_lag_1
едем дальше
в итоге получили список ['month_lag_1', 'cnt_WMA_3_lag_1', 'weekday_Friday_lag_1', 'year_lag_1']
запуск первого шага
new var_for_add ! weekday_Sunday_lag_1
едем дальше
в итоге получили список ['month_lag_1', 'cnt_WMA_3_lag_1', 'weekday_Friday_lag_1', 'year_lag_1', 'weekday_Sunday_lag_1']
запуск первого шага
мы сошлись
['month_lag_1', 'cnt_WMA_3_lag_1', 'weekday_Friday_lag_1', 'year_lag_1', 'weekday_Sunday_lag_1']
0.424



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [31]:
data_prediction

,month_lag_1,cnt_WMA_3_lag_1,weekday_Friday_lag_1,year_lag_1,weekday_Sunday_lag_1,mean,cnt_SMA_3_lag_1,item_id,date,cnt,model_prediction
80000,12.0,12.000000,0.0,2015.0,0.0,16.5,13.333333,STORE_1_714,2015-12-17,20.0,16.257999
80001,12.0,15.333333,0.0,2015.0,0.0,16.5,13.666667,STORE_1_714,2015-12-18,20.0,19.190921
80002,12.0,18.500000,1.0,2015.0,0.0,16.5,17.000000,STORE_1_714,2015-12-19,34.0,22.528674
80003,12.0,27.000000,0.0,2015.0,0.0,16.5,24.666667,STORE_1_714,2015-12-20,12.0,23.870809
80004,12.0,20.666667,0.0,2015.0,1.0,16.5,22.000000,STORE_1_714,2015-12-21,10.0,18.217096
80005,12.0,14.666667,0.0,2015.0,0.0,16.5,18.666667,STORE_1_714,2015-12-22,26.0,19.190921
80006,12.0,18.333333,0.0,2015.0,0.0,16.5,16.000000,STORE_1_714,2015-12-23,25.0,19.138627
80007,12.0,22.833333,0.0,2015.0,0.0,16.5,20.333333,STORE_1_714,2015-12-24,15.0,20.393423
80008,12.0,20.166667,0.0,2015.0,0.0,16.5,22.000000,STORE_1_714,2015-12-25,1.0,19.278077
80009,12.0,9.666667,1.0,2015.0,0.0,16.5,13.666667,STORE_1_714,2015-12-26,14.0,18.929496


In [17]:
horizont = 1

In [18]:
technical_list = ['index_time', 'item_id', 'store_id', 'date_id','cnt', 'date', 'wm_yr_wk', 'wday']

In [19]:
list_for_lags  = [i for i in df_m.columns if i not in technical_list]

In [20]:
lags, pred_df = create_lag_features_with_prediction(df_m, list_for_lags, horizont, horizont)

In [23]:
lags

,item_id,store_id,date_id,cnt,date,wm_yr_wk,wday,month,year,CASHBACK_STORE_1,...,weekday_Friday_lag_1,weekday_Monday_lag_1,weekday_Saturday_lag_1,weekday_Sunday_lag_1,weekday_Thursday_lag_1,weekday_Tuesday_lag_1,weekday_Wednesday_lag_1,cnt_SMA_3_lag_1,cnt_WMA_3_lag_1,cnt_EMA_3_lag_1
78218,STORE_1_714,STORE_1,2,38,2011-01-30,11101,2,1,2011,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,38.000000,41.166667,40.662406
78219,STORE_1_714,STORE_1,3,28,2011-01-31,11101,3,1,2011,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,41.333333,41.166667,39.331203
78220,STORE_1_714,STORE_1,4,27,2011-02-01,11101,4,2,2011,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,37.666667,34.500000,33.665602
78221,STORE_1_714,STORE_1,5,23,2011-02-02,11101,5,2,2011,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,31.000000,29.166667,30.332801
78222,STORE_1_714,STORE_1,6,32,2011-02-03,11101,6,2,2011,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,26.000000,25.166667,26.666400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80031,STORE_1_714,STORE_1,1815,18,2016-01-17,11551,2,1,2016,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,16.000000,17.000000,17.239804
80032,STORE_1_714,STORE_1,1816,10,2016-01-18,11551,3,1,2016,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,17.666667,18.000000,17.619902
80033,STORE_1_714,STORE_1,1817,12,2016-01-19,11551,4,1,2016,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,15.666667,14.166667,13.809951
80034,STORE_1_714,STORE_1,1818,11,2016-01-20,11551,5,1,2016,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,13.333333,12.333333,12.904975


In [24]:
lags_cols = [i for i in lags.columns if i not in df_m.columns]

In [25]:
df_m = pd.concat([df_m, lags[lags_cols]], axis = 1)

In [26]:
from Tools import *

In [27]:
# Задаем значение для генератора случайных чисел
seed_value = 23
np.random.seed(seed_value)

In [28]:
df_m.date

78217   2011-01-29
78218   2011-01-30
78219   2011-01-31
78220   2011-02-01
78221   2011-02-02
           ...    
80031   2016-01-17
80032   2016-01-18
80033   2016-01-19
80034   2016-01-20
80035   2016-01-21
Name: date, Length: 1819, dtype: datetime64[ns]

In [29]:
# creating cross validator
window_m = len(df_m.date.unique())//5
test_m = window_m//5
cv_datetime = DateTimeSeriesSplit(window = window_m, n_splits= 4, test_size = test_m, margin=0)
group_dt = df_m['date']

In [30]:
## get parametres from train and test for each fold
for fold, (train_idx, val_idx) in enumerate(cv_datetime.split(df_m, groups=group_dt), 1):
    print(fold)
    train, test = df_m.iloc[train_idx], df_m.iloc[val_idx]
    print(f'треин мин {train.date.min()} треин макс {train.date.max()} shape {train.shape}')
    print(f'тест мин {test.date.min()} тест макс {test.date.max()} shape {test.shape}')

1
треин мин 2014-04-11 00:00:00 треин макс 2015-04-08 00:00:00 shape (363, 120)
тест мин 2015-04-09 00:00:00 тест макс 2015-06-19 00:00:00 shape (72, 120)
2
треин мин 2014-06-22 00:00:00 треин макс 2015-06-19 00:00:00 shape (363, 120)
тест мин 2015-06-20 00:00:00 тест макс 2015-08-30 00:00:00 shape (72, 120)
3
треин мин 2014-09-02 00:00:00 треин макс 2015-08-30 00:00:00 shape (363, 120)
тест мин 2015-08-31 00:00:00 тест макс 2015-11-10 00:00:00 shape (72, 120)
4
треин мин 2014-11-13 00:00:00 треин макс 2015-11-10 00:00:00 shape (363, 120)
тест мин 2015-11-11 00:00:00 тест макс 2016-01-21 00:00:00 shape (72, 120)


In [31]:
# create model for selector
from lightgbm import LGBMRegressor
model = LGBMRegressor(max_depth=3, verbosity = -1)

Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.

Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.



In [32]:
# import metrics for var selection
from sklearn.metrics import mean_absolute_percentage_error as MAPE

# create selector
selector1 = Kraken(model, cv_datetime, MAPE, 'exp1')

In [33]:
# get rank dict from vars
selector1.get_rank_dict(df_m, df_m['cnt'], lags_cols, group_dt)


In [34]:
selector1.fe_dict

{'month_lag_1': 7.606025815246571,
 'year_lag_1': 1.785581836483179,
 'CASHBACK_STORE_1_lag_1': 0.3797136774864408,
 'CASHBACK_STORE_2_lag_1': 0.711591099741111,
 'CASHBACK_STORE_3_lag_1': 0.3632236490418903,
 'sell_price_lag_1': 0.0,
 'event_name_1_Chanukah End_lag_1': 0.0,
 'event_name_1_Christmas_lag_1': 0.0,
 'event_name_1_Cinco De Mayo_lag_1': 0.0,
 'event_name_1_ColumbusDay_lag_1': 0.0,
 'event_name_1_Easter_lag_1': 0.0,
 'event_name_1_Eid al-Fitr_lag_1': 0.0,
 'event_name_1_EidAlAdha_lag_1': 0.0,
 "event_name_1_Father's day_lag_1": 0.0,
 'event_name_1_Halloween_lag_1': 0.0,
 'event_name_1_IndependenceDay_lag_1': 0.0,
 'event_name_1_LaborDay_lag_1': 0.0,
 'event_name_1_LentStart_lag_1': 0.0,
 'event_name_1_LentWeek2_lag_1': 0.0,
 'event_name_1_MartinLutherKingDay_lag_1': 0.0,
 'event_name_1_MemorialDay_lag_1': 0.0,
 "event_name_1_Mother's day_lag_1": 0.0,
 'event_name_1_NBAFinalsEnd_lag_1': 0.0,
 'event_name_1_NBAFinalsStart_lag_1': 0.0,
 'event_name_1_NewYear_lag_1': 0.0,
 'even

In [35]:
## get vars
vars_final = selector1.get_vars(df_m, df_m['cnt'], early_stopping_rounds = 100, group_dt = group_dt); vars_final

запуск первого шага
new var_for_add ! month_lag_1
едем дальше
в итоге получили список ['month_lag_1']
запуск первого шага
new var_for_add ! cnt_EMA_3_lag_1
new var_for_add ! cnt_WMA_3_lag_1
едем дальше
в итоге получили список ['month_lag_1', 'cnt_WMA_3_lag_1']
запуск первого шага
new var_for_add ! weekday_Friday_lag_1
едем дальше
в итоге получили список ['month_lag_1', 'cnt_WMA_3_lag_1', 'weekday_Friday_lag_1']
запуск первого шага
new var_for_add ! year_lag_1
едем дальше
в итоге получили список ['month_lag_1', 'cnt_WMA_3_lag_1', 'weekday_Friday_lag_1', 'year_lag_1']
запуск первого шага
new var_for_add ! weekday_Sunday_lag_1
едем дальше
в итоге получили список ['month_lag_1', 'cnt_WMA_3_lag_1', 'weekday_Friday_lag_1', 'year_lag_1', 'weekday_Sunday_lag_1']
запуск первого шага
мы сошлись
['month_lag_1', 'cnt_WMA_3_lag_1', 'weekday_Friday_lag_1', 'year_lag_1', 'weekday_Sunday_lag_1']
0.424


['month_lag_1',
 'cnt_WMA_3_lag_1',
 'weekday_Friday_lag_1',
 'year_lag_1',
 'weekday_Sunday_lag_1']

In [36]:
if len(vars_final) == 0:
    vars_final = [i for i in list_in_model if i.startswith('cnt')]

In [37]:
vars_final

['month_lag_1',
 'cnt_WMA_3_lag_1',
 'weekday_Friday_lag_1',
 'year_lag_1',
 'weekday_Sunday_lag_1']

In [38]:
test_dates = pd.Series(df_m['date'].unique()).sort_values().tail(max(3, test_m//2)).values

In [39]:
test_dates

array(['2015-12-17T00:00:00.000000000', '2015-12-18T00:00:00.000000000',
       '2015-12-19T00:00:00.000000000', '2015-12-20T00:00:00.000000000',
       '2015-12-21T00:00:00.000000000', '2015-12-22T00:00:00.000000000',
       '2015-12-23T00:00:00.000000000', '2015-12-24T00:00:00.000000000',
       '2015-12-25T00:00:00.000000000', '2015-12-26T00:00:00.000000000',
       '2015-12-27T00:00:00.000000000', '2015-12-28T00:00:00.000000000',
       '2015-12-29T00:00:00.000000000', '2015-12-30T00:00:00.000000000',
       '2015-12-31T00:00:00.000000000', '2016-01-01T00:00:00.000000000',
       '2016-01-02T00:00:00.000000000', '2016-01-03T00:00:00.000000000',
       '2016-01-04T00:00:00.000000000', '2016-01-05T00:00:00.000000000',
       '2016-01-06T00:00:00.000000000', '2016-01-07T00:00:00.000000000',
       '2016-01-08T00:00:00.000000000', '2016-01-09T00:00:00.000000000',
       '2016-01-10T00:00:00.000000000', '2016-01-11T00:00:00.000000000',
       '2016-01-12T00:00:00.000000000', '2016-01-13

In [40]:
X_train = df_m[~df_m['date'].isin(test_dates)]
y_train = df_m[~df_m['date'].isin(test_dates)]['cnt']

# oot - out of time
X_oot = df_m[df_m['date'].isin(test_dates)]
y_oot = df_m[df_m['date'].isin(test_dates)]['cnt']

LGBMRegressor(max_depth=3, verbosity=-1)

In [42]:
model.fit(X_train[vars_final], y_train)
pred_df['date'] = X_oot.date.max() + pd.Timedelta(days=horizont)
pred_df['cnt'] = np.nan
pred_df['item_id'] = X_oot.item_id.max()
pred_df['mean'] = X_oot.cnt.mean()
X_oot['mean'] = X_oot.cnt.mean()
list_cont = vars_final + ['mean','cnt_SMA_3_lag_1','item_id', 'date','cnt']
data_prediction = pd.concat([X_oot[list_cont], pred_df[list_cont]], axis = 0)
data_prediction['model_prediction'] = model.predict(data_prediction[vars_final])


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [50]:
list_cont = vars_final + ['mean','cnt_SMA_3_lag_1','item_id', 'date','cnt']

In [51]:
data_prediction = pd.concat([X_oot[list_cont], pred_df[list_cont]], axis = 0)

In [52]:
data_prediction['model_prediction'] = model.predict(data_prediction[vars_final])

In [53]:
data_prediction

,month_lag_1,cnt_WMA_3_lag_1,weekday_Friday_lag_1,year_lag_1,weekday_Sunday_lag_1,mean,cnt_SMA_3_lag_1,item_id,date,cnt,model_prediction
80000,12.0,12.000000,0.0,2015.0,0.0,16.5,13.333333,STORE_1_714,2015-12-17,20.0,16.916591
80001,12.0,15.333333,0.0,2015.0,0.0,16.5,13.666667,STORE_1_714,2015-12-18,20.0,20.208089
80002,12.0,18.500000,1.0,2015.0,0.0,16.5,17.000000,STORE_1_714,2015-12-19,34.0,23.130954
80003,12.0,27.000000,0.0,2015.0,0.0,16.5,24.666667,STORE_1_714,2015-12-20,12.0,23.677227
80004,12.0,20.666667,0.0,2015.0,1.0,16.5,22.000000,STORE_1_714,2015-12-21,10.0,18.404998
80005,12.0,14.666667,0.0,2015.0,0.0,16.5,18.666667,STORE_1_714,2015-12-22,26.0,20.208089
80006,12.0,18.333333,0.0,2015.0,0.0,16.5,16.000000,STORE_1_714,2015-12-23,25.0,19.607651
80007,12.0,22.833333,0.0,2015.0,0.0,16.5,20.333333,STORE_1_714,2015-12-24,15.0,20.627784
80008,12.0,20.166667,0.0,2015.0,0.0,16.5,22.000000,STORE_1_714,2015-12-25,1.0,19.586574
80009,12.0,9.666667,1.0,2015.0,0.0,16.5,13.666667,STORE_1_714,2015-12-26,14.0,19.180202
